In [1]:
# Investigating which classes suffer from test-time augmentation,
# and why
import sys
sys.path.insert(0, '../')
from dataloaders import get_imnet_dataloader
from utils.gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch('0')

import h5py
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from imagenet_classes import imagenet_classes


sns.set_style('white')
hf = h5py.File('../imnet/five_crop_hflip_scale/model_outputs/val/resnet50.h5', 'r')
outputs = []
labels = []
for key in hf.keys():
    if 'inputs' in key:
        batch_key = key
        label_key = key[:-7] + "_labels"
        outputs.append(hf[batch_key][:])
        labels.append(hf[label_key][:])
outputs = np.concatenate(outputs, axis=1)
labels = np.concatenate(labels)

Using GPU:0


In [6]:
from sklearn.model_selection import train_test_split
train_idxs, test_idxs = train_test_split(list(range(outputs.shape[1])), train_size=.5)
train_outputs, train_labels = outputs[:,train_idxs,:], labels[train_idxs]
test_outputs, test_labels = outputs[:,test_idxs,:], labels[train_idxs]

In [20]:
# identify which predicted classes are hurt by standard TTA
orig_idx = 12
stta_preds = np.mean(train_outputs, axis=0)
orig_preds = train_outputs[orig_idx,:,:]

stta_correct_idxs = np.where(np.argmax(stta_preds, axis=1) == train_labels)[0]
stta_incorrect_idxs = np.where(np.argmax(stta_preds, axis=1) != train_labels)[0]
orig_correct_idxs = np.where(np.argmax(orig_preds, axis=1) == train_labels)[0]
orig_incorrect_idxs = np.where(np.argmax(orig_preds, axis=1) != train_labels)[0]

corruptions = list(set(stta_incorrect_idxs).intersection(orig_correct_idxs))
corrections = list(set(stta_correct_idxs).intersection(orig_incorrect_idxs))

In [22]:
corrupt_predicted_classes = np.argmax(stta_preds[corruptions], axis= 1)
correct_predicted_classes = np.argmax(stta_preds[corrections], axis= 1)

In [27]:
corrupt_map = np.zeros(1000)
correct_map = np.zeros(1000)
for j in corrupt_predicted_classes:
    corrupt_map[j] += 1

for j in correct_predicted_classes:
    correct_map[j] += 1

In [30]:
diffs = correct_map - corrupt_map
classes_to_avoid = np.where(diffs < 0)[0]

In [31]:
combined_preds = []
for i in range(len(stta_preds)):
    stta_pred = stta_preds[i]
    orig_pred = orig_preds[i]
    if np.argmax(stta_pred) in classes_to_avoid:
        combined_preds.append()
# iterate through predictions from stta and predictions from orig
# if np.argmax(stta_pred).in(classes_to_avoid)
# add orig_pred
# if not, add stta_pred

195

In [32]:
stta_preds.shape

(24999, 1000)